### Importando as bibliotecas

In [1]:
import pandas as pd


### Importando o dataset

In [5]:
colunas = ['date', 'time', 'open', 'high', 'low', 'close', 'volume']
df = pd.read_csv('../dataset/dataset-exemplo-202206.csv')
df.columns = colunas

In [6]:
df.head()

,date,time,open,high,low,close,volume
0,2022.06.01,00:01,1.07181,1.07183,1.07177,1.07181,0
1,2022.06.01,00:02,1.07182,1.07183,1.07172,1.07172,0
2,2022.06.01,00:03,1.07172,1.07172,1.07157,1.07158,0
3,2022.06.01,00:04,1.07157,1.07160,1.07151,1.07160,0
4,2022.06.01,00:05,1.07162,1.07163,1.07157,1.07162,0


### Verificando valores omissos

In [7]:
df.isnull().sum()

date      0
time      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64